In [ ]:
import numpy as np
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from __future__ import annotations
import os
import logging
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import time

import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import transformers
# https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/excel.html?highlight=xlsx#microsoft-excel
from langchain.document_loaders import (
    CSVLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
)

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS

[nltk_data] Error loading omw: <urlopen error [Errno 101] Network is
[nltk_data]     unreachable>


In [2]:
import os
INGEST_THREADS = os.cpu_count() or 8
def load_single_document(file_path: str) -> Document:
    # Loads a single document from a file path
    file_extension = os.path.splitext(file_path)[1]
    loader_class = DOCUMENT_MAP.get(file_extension)
    if loader_class == TextLoader:
        loader = TextLoader(file_path, encoding="utf-8")
    elif loader_class:
        loader = loader_class(file_path)
    else:
        raise ValueError("Document type is undefined")
    return loader.load()[0]
DOCUMENT_MAP = {
    ".txt": TextLoader,
    ".md": TextLoader,
    ".pdf": PDFMinerLoader,
    ".csv": CSVLoader,
    ".xls": UnstructuredExcelLoader,
    ".xlsx": UnstructuredExcelLoader,
    ".docx": Docx2txtLoader,
    ".doc": Docx2txtLoader,
}
def load_document_batch(filepaths):
    logging.info("Loading document batch")
    # create a thread pool
    with ThreadPoolExecutor(len(filepaths)) as exe:
        # load files
        futures = [exe.submit(load_single_document, name) for name in filepaths]
        # collect data
        data_list = [future.result() for future in futures]
        # return data and file paths
        return (data_list, filepaths)
def loadDocuments(
    source_dir: str, chunk_size=1000, chunk_overlap=200
) -> list[Document]:
    # Loads all documents from the source documents directory, including nested folders
    paths = []
    for root, _, files in os.walk(source_dir):
        for file_name in files:
            file_extension = os.path.splitext(file_name)[1]
            source_file_path = os.path.join(root, file_name)
            if file_extension in DOCUMENT_MAP.keys():
                paths.append(source_file_path)

    # Have at least one worker and at most INGEST_THREADS workers
    n_workers = min(INGEST_THREADS, max(len(paths), 1))
    # chunksize = round(len(paths) / n_workers)
    chunksize = max(round(len(paths) / n_workers), 1)
    docs = []
    with ProcessPoolExecutor(n_workers) as executor:
        futures = []
        # split the load operations into chunks
        for i in range(0, len(paths), chunksize):
            # select a chunk of filenames
            filepaths = paths[i : (i + chunksize)]
            # submit the task
            future = executor.submit(load_document_batch, filepaths)
            futures.append(future)
        # process all results
        for future in as_completed(futures):
            # open the file and load the data
            contents, _ = future.result()
            docs.extend(contents)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    documents: list[Document]
    documents = text_splitter.split_documents(docs)
    # documents = char_data_splitter(docs, chunk_size, chunk_overlap)
    return documents


In [3]:
endl = "\n"

In [4]:
#load_dotenv()
import re
# root_dir = "."
# sys.path.append(root_dir)  # if import module in this project error
if os.name != "nt":
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
# %% [markdown]
###**setup var**

#%%
#chunk_size = 2000
# chunk_overlap = 200
# embedding_algorithm = "faiss"
# source_directory = f"{root_dir}/ir-service/docs"
# persist_directory = f"{root_dir}/ir-service/tmp/embeddings/{embedding_algorithm}"
# print(root_dir)
# print(persist_directory)

# Original mapper dictionary
mapper = {
    "law_doc-84-89.txt": "761/2566",
    "law_doc-44-46.txt": "1301/2566",
    "law_doc-54-57.txt": "1225/2566",
    "law_doc-12-13.txt": "2525/2566",
    "law_doc-40-43.txt": "1305/2566",
    "law_doc-14-15.txt": "2085/2566",
    "law_doc-64-69.txt": "1090/2566",
    "law_doc-1-5.txt": "2610/2566",
    "law_doc-78-81.txt": "882/2566",
    "law_doc-82-83.txt": "835/2566",
    "law_doc-35-39.txt": "1306/2566",
    "law_doc-16-20.txt": "1574/2566",
    "law_doc-32-34.txt": "1373/2566",
    "law_doc-74-77.txt": "934/2566",
    "law_doc-6-11.txt": "2609/2566",
    "law_doc-90-92.txt": "756/2566",
    "law_doc-47-53.txt": "1300/2566",
    "law_doc-58-63.txt": "1101/2566",
    "law_doc-70-73.txt": "1003/2566",
    "law_doc-21-31.txt": "1542/2566",
}

# Reverse the mapping and format it
mapper_reverse = {f"คดี {case}": filename for filename, case in mapper.items()}

endl = "\n"
# print(root_dir)
exclude_pattern = re.compile(r"[^ก-๙]+")  # |[^0-9a-zA-Z]+

In [5]:
import sys
root_dir = os.path.dirname(os.getcwd())
# sys.path.append(os.path.join(root_dir, 'deployment/ir-trt-service'))

In [6]:
contents = None
documents_specific = None
documents_general = None
documents = None


if contents is None:
    with open("/home/shanwibo/Capstone-TamTanai/notebooks/specific_case_knowledge.txt", "r", encoding="utf-8") as f:
        content = f.read()
    contents = content.split("\n\n")

    if documents_specific is None:
        documents_specific = [
            Document(
                page_content=f"{endl.join(c.split(endl)[1:])}",
                metadata={
                    "source": f"docs/{mapper_reverse[c.split(endl)[0]]}",
                    "category": "specific",
                },
            )
            for c in contents
        ]
if documents_general is None:
    documents_general = loadDocuments(
            source_dir=f"/home/shanwibo/Capstone-TamTanai/asset/documentation", chunk_size=10e14, chunk_overlap=0
        )
    for i in range(len(documents_general)):
        documents_general[i].metadata["source"] = (
                documents_general[i].metadata["source"].replace(f"{root_dir}/", "")
            )
        documents_general[i].metadata["category"] = "general"
if documents is None:
    documents = documents_general + documents_specific

In [7]:
documents_general[0]

Document(metadata={'source': '/home/shanwibo/Capstone-TamTanai/asset/documentation/พระราชบัญญัติเงินทดแทน/พระราชบัญญัติเงินทดแทน_หมวด5.txt', 'category': 'general'}, page_content='พระราชบัญญัติเงินทดแทน (ฉบับที่ 2) พ.ศ. 2561 - หมวด 5 (เงินสมทบ)\n\nมาตรา 44  ให้กระทรวงแรงงานประกาศกำหนดประเภทและขนาดของกิจการ และท้องที่ที่นายจ้างต้องจ่ายเงินสมทบ\nให้นายจ้างซึ่งมีหน้าที่ต้องจ่ายเงินสมทบตามวรรคหนึ่งต้องดำเนินการ ดังต่อไปนี้ ภายในสามสิบวันนับแต่วันที่นายจ้างมีหน้าที่ต้องจ่ายเงินสมทบ\n(1) ยื่นแบบรายการขึ้นทะเบียนนายจ้าง และ\n(2) จ่ายเงินสมทบ\nกรณีข้อเท็จจริงในแบบรายการขึ้นทะเบียนนายจ้างเปลี่ยนแปลงไป ให้นายจ้างแจ้งการเปลี่ยนแปลงภายในวันที่สิบห้าของเดือนถัดจากเดือนที่มีการเปลี่ยนแปลง\nแบบรายการ วิธีการยื่นแบบรายการ การจ่ายเงินสมทบ และการแจ้งการเปลี่ยนแปลงแบบรายการให้เป็นไปตามที่เลขาธิการประกาศกำหนด\n\nมาตรา 45  เพื่อประโยชน์ในการเรียกเก็บเงินสมทบจากนายจ้างตามมาตรา 44 ให้กระทรวงแรงงาน* มีอำนาจประกาศกำหนดอัตราเงินสมทบไม่เกินร้อยละห้าของค่าจ้างที่นายจ้างจ่ายแต่ละปี อัตราเงินฝากสำหรับกรณีที่นายจ้างข

In [8]:
####สร้าง Dataset ที่ Negative มีเเต่ประมวลกฎหมายอื่น

In [9]:
import pandas as pd
df = pd.read_csv("../asset/dataset/dataset.csv")
df['source'] = df['source'].str.replace("_ตรวจแล้ว/", "", regex=False)
df

,question,answer,references,source,knowledges
0,ศาลมีอำนาจใดในการพิจารณาคดีเกี่ยวกับยาเสพติดตา...,ศาลมีอำนาจพิจารณาพิพากษาคดีโดยคำนึงถึงการสงเคร...,มาตรา 165 ของประมวลกฎหมายยาเสพติด,ประมวลกฎหมายยาเสพติด/ประมวลกฎหมายยาเสพติด_ภาค3...,ประมวลกฎหมายยาเสพติด - ภาค 3 (บทกำหนดโทษ) - ลั...
1,เมื่อไหร่ที่สมาคมนายจ้างจะถือว่าเลิก,สมาคมนายจ้างย่อมเลิกด้วยเหตุใดเหตุหนึ่ง ดังต่อ...,มาตรา 82,พระราชบัญญัติแรงงานสัมพันธ์/พระราชบัญญัติแรงงา...,พระราชบัญญัติแรงงานสัมพันธ์ (ฉบับที่ 3) พ.ศ. 2...
2,การยื่นอุทธรณ์ต้องทำภายในกี่เดือนหลังจากอ่านหร...,ต้องยื่นต่อศาลชั้นต้นในกำหนดหนึ่งเดือน,มาตรา 198 ของประมวลกฎหมายวิธีพิจารณาความอาญา,ประมวลกฎหมายวิธีพิจารณาความอาญา/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความอาญา - ภาค 4 (อุทธร...
3,ศาลมีอำนาจใดบ้างเกี่ยวกับการรับฟังพยานหลักฐาน?,ศาลมีอำนาจปฏิเสธไม่รับพยานหลักฐานที่รับฟังไม่ไ...,อ้างอิงจากมาตรา 86 ของประมวลกฎหมายวิธีพิจารณาค...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...
4,หากผู้เช่านาต้องการอุทธรณ์คำวินิจฉัยของคณะกรรม...,ผู้เช่านาต้องทำเป็นหนังสือยื่นต่อประธานคณะกรรม...,อ้างอิงจากมาตรา 56 ของพระราชบัญญัติการเช่าที่ด...,พระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม/พระรา...,พระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม พ.ศ. ...
...,...,...,...,...,...
4116,เมื่อเจ้าหน้าที่ของรัฐพบว่ามีการกระทำความผิดทา...,เจ้าหน้าที่ของรัฐจะต้องดำเนินการแสวงหาข้อเท็จจ...,มาตรา 19 และ 20 ของพระราชบัญญัติว่าด้วยการปรับ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
4117,ข้อพิพาทแรงงานหมายถึงอะไรตามพระราชบัญญัติแรงงา...,หมายถึงข้อขัดแย้งระหว่างนายจ้างกับลูกจ้างเกี่ย...,มาตรา 5 ของพระราชบัญญัติแรงงานสัมพันธ์ พ.ศ. 2518,พระราชบัญญัติแรงงานสัมพันธ์/พระราชบัญญัติแรงงา...,พระราชบัญญัติแรงงานสัมพันธ์ พ.ศ. 2518\n\nมาตรา...
4118,หากกรรมการผู้แทนองค์กรลูกจ้างหรือกรรมการผู้ทรง...,"ใช่, จะถูกพ้นจากตำแหน่งหากรัฐมนตรีให้ออกเพราะบ...",มาตรา 20 ของพระราชกำหนดการบริหารจัดการการทำงาน...,พระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้า...,พระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้า...
4119,เมื่อหนี้และสิทธิในการรับหนี้ตกอยู่กับบุคคลเดี...,ตามมาตรา 353 ของประมวลกฎหมายแพ่งและพาณิชย์ หนี...,อ้างอิงจากมาตรา 353 ของประมวลกฎหมายแพ่งและพาณิชย์,ประมวลกฎหมายแพ่งและพาณิชย์/ประมวลกฎหมายแพ่งและ...,ประมวลกฎหมายแพ่งและพาณิชย์ - บรรพ 2 (หนี้) - ล...


In [35]:
from tqdm import tqdm #ไม่เอาทุกกฏหมายอื่น
check = []
for i in tqdm(range(len(df))) :
  check.append({"query": df.iloc[i,0], "pos": [documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3] in documents_general[e].metadata["source"]], "neg":[documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3].split("/")[0] not in documents_general[e].metadata["source"]]})

100%|██████████| 4121/4121 [00:58<00:00, 71.04it/s]


In [36]:
import json
from tqdm import tqdm
data_name = "reranker_training_dataset_without_other_law.jsonl"
output_path = f"/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{data_name}"
for i in tqdm(range(len(check))) :
  with open(output_path, "a", encoding='utf-8') as final:
      json.dump(check[i], final, ensure_ascii=False)
      final.write('\n')

100%|██████████| 4121/4121 [01:17<00:00, 53.42it/s]


In [37]:
####ไม่เอาเอกสารอื่นทั้งหมด

In [38]:
from tqdm import tqdm 
check = []
for i in tqdm(range(len(df))) :
  check.append({"query": df.iloc[i,0], "pos": [documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3] in documents_general[e].metadata["source"]], "neg":[documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3] not in documents_general[e].metadata["source"]]})

100%|██████████| 4121/4121 [00:56<00:00, 73.01it/s]


In [39]:
import json
from tqdm import tqdm
data_name = "reranker_training_dataset_without_other_documents.jsonl"
output_path = f"/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{data_name}"
for i in tqdm(range(len(check))) :
  with open(output_path, "a", encoding='utf-8') as final:
      json.dump(check[i], final, ensure_ascii=False)
      final.write('\n')

100%|██████████| 4121/4121 [01:20<00:00, 51.27it/s]


In [40]:
####เเบบสุ่ม Negative มา n ตัว (สุ่มจากประมวลกฎหมายอื่น)

In [20]:
from tqdm import tqdm 
from transformers import set_seed
import random

main_seed = 277
num = 10
check = []

for i in tqdm(range(len(df))) :
  a = [documents_general[q].page_content for q in range(len(documents_general)) if df.iloc[i, 3].split("/")[0] not in documents_general[q].metadata["source"]]
  random.seed(main_seed)
  set_seed(main_seed)
  random_numbers = [random.randint(0, len(a)-1) for _ in range(num)]
  check.append({"query": df.iloc[i,0], "pos": [documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3] in documents_general[e].metadata["source"]], "neg": [a[m] for m in random_numbers]})

100%|██████████| 4121/4121 [01:02<00:00, 66.07it/s]


In [74]:
from tqdm import tqdm 
from transformers import set_seed
import random

main_seed = 277
num = 10
check = []

for i in tqdm(range(10)) :
  a = [documents_general[q].page_content for q in range(len(documents_general)) if df.iloc[i, 3].split("/")[0] not in documents_general[q].metadata["source"]]
  random.seed(main_seed)
  set_seed(main_seed)
  random_numbers = [random.randint(0, len(a)-1) for _ in range(num)]
  check.append({"query": df.iloc[i,0], "pos": [documents_general[e].page_content for e in range(len(documents_general)) if df.iloc[i, 3] in documents_general[e].metadata["source"]], "neg": [a[m] for m in random_numbers]})

100%|██████████| 10/10 [00:00<00:00, 66.59it/s]


In [21]:
import json
from tqdm import tqdm
data_name = "reranker_training_dataset_with_Negative=10.jsonl"
output_path = f"/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{data_name}"
for i in tqdm(range(len(check))) :
  with open(output_path, "a", encoding='utf-8') as final:
      json.dump(check[i], final, ensure_ascii=False)
      final.write('\n')

100%|██████████| 4121/4121 [00:04<00:00, 960.80it/s]


# Revised

In [18]:
import os
import glob
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [2]:
seed = 0

In [3]:
DOCUMENT_PREFIX_DIR = '/home/shanwibo/Capstone-TamTanai/asset/documentation'

In [4]:
df = pd.read_csv("../asset/dataset/dataset.csv")
df['source'] = df['source'].str.replace("_ตรวจแล้ว/", "", regex=False)
df

,question,answer,references,source,knowledges
0,ศาลมีอำนาจใดในการพิจารณาคดีเกี่ยวกับยาเสพติดตา...,ศาลมีอำนาจพิจารณาพิพากษาคดีโดยคำนึงถึงการสงเคร...,มาตรา 165 ของประมวลกฎหมายยาเสพติด,ประมวลกฎหมายยาเสพติด/ประมวลกฎหมายยาเสพติด_ภาค3...,ประมวลกฎหมายยาเสพติด - ภาค 3 (บทกำหนดโทษ) - ลั...
1,เมื่อไหร่ที่สมาคมนายจ้างจะถือว่าเลิก,สมาคมนายจ้างย่อมเลิกด้วยเหตุใดเหตุหนึ่ง ดังต่อ...,มาตรา 82,พระราชบัญญัติแรงงานสัมพันธ์/พระราชบัญญัติแรงงา...,พระราชบัญญัติแรงงานสัมพันธ์ (ฉบับที่ 3) พ.ศ. 2...
2,การยื่นอุทธรณ์ต้องทำภายในกี่เดือนหลังจากอ่านหร...,ต้องยื่นต่อศาลชั้นต้นในกำหนดหนึ่งเดือน,มาตรา 198 ของประมวลกฎหมายวิธีพิจารณาความอาญา,ประมวลกฎหมายวิธีพิจารณาความอาญา/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความอาญา - ภาค 4 (อุทธร...
3,ศาลมีอำนาจใดบ้างเกี่ยวกับการรับฟังพยานหลักฐาน?,ศาลมีอำนาจปฏิเสธไม่รับพยานหลักฐานที่รับฟังไม่ไ...,อ้างอิงจากมาตรา 86 ของประมวลกฎหมายวิธีพิจารณาค...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...
4,หากผู้เช่านาต้องการอุทธรณ์คำวินิจฉัยของคณะกรรม...,ผู้เช่านาต้องทำเป็นหนังสือยื่นต่อประธานคณะกรรม...,อ้างอิงจากมาตรา 56 ของพระราชบัญญัติการเช่าที่ด...,พระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม/พระรา...,พระราชบัญญัติการเช่าที่ดินเพื่อเกษตรกรรม พ.ศ. ...
...,...,...,...,...,...
4116,เมื่อเจ้าหน้าที่ของรัฐพบว่ามีการกระทำความผิดทา...,เจ้าหน้าที่ของรัฐจะต้องดำเนินการแสวงหาข้อเท็จจ...,มาตรา 19 และ 20 ของพระราชบัญญัติว่าด้วยการปรับ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
4117,ข้อพิพาทแรงงานหมายถึงอะไรตามพระราชบัญญัติแรงงา...,หมายถึงข้อขัดแย้งระหว่างนายจ้างกับลูกจ้างเกี่ย...,มาตรา 5 ของพระราชบัญญัติแรงงานสัมพันธ์ พ.ศ. 2518,พระราชบัญญัติแรงงานสัมพันธ์/พระราชบัญญัติแรงงา...,พระราชบัญญัติแรงงานสัมพันธ์ พ.ศ. 2518\n\nมาตรา...
4118,หากกรรมการผู้แทนองค์กรลูกจ้างหรือกรรมการผู้ทรง...,"ใช่, จะถูกพ้นจากตำแหน่งหากรัฐมนตรีให้ออกเพราะบ...",มาตรา 20 ของพระราชกำหนดการบริหารจัดการการทำงาน...,พระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้า...,พระราชกำหนดการบริหารจัดการการทำงานของคนต่างด้า...
4119,เมื่อหนี้และสิทธิในการรับหนี้ตกอยู่กับบุคคลเดี...,ตามมาตรา 353 ของประมวลกฎหมายแพ่งและพาณิชย์ หนี...,อ้างอิงจากมาตรา 353 ของประมวลกฎหมายแพ่งและพาณิชย์,ประมวลกฎหมายแพ่งและพาณิชย์/ประมวลกฎหมายแพ่งและ...,ประมวลกฎหมายแพ่งและพาณิชย์ - บรรพ 2 (หนี้) - ล...


In [5]:
document_df = []
for document_path in glob.glob(os.path.join(DOCUMENT_PREFIX_DIR, '*/*.txt')):
    with open(document_path) as file:
        document_df.append({
            'source': '/'.join(document_path.split('/')[-2:]),
            'content': file.read()
        })
document_df = pd.DataFrame(document_df)
document_df

,source,content
0,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
1,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
2,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
3,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...
4,พระราชบัญญัติจราจรทางบก/พระราชบัญญัติจราจรทางบ...,พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565...
...,...,...
562,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 4 (วิธีก...
563,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...
564,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 2 (วิธีพ...
565,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...


# เเบบสุ่ม Negative มา n ตัว (สุ่มจากประมวลกฎหมายอื่น)

## 1

In [17]:
n_negative = 1
np.random.seed(seed)

dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': document_df[document_df['source'] != row['source']].sample(n=n_negative, random_state=seed)['content'].tolist()
    })

100%|██████████| 4121/4121 [00:02<00:00, 1518.22it/s]


In [18]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 8896.18it/s]


## 5

In [19]:
n_negative = 5
np.random.seed(seed)

dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': document_df[document_df['source'] != row['source']].sample(n=n_negative, random_state=seed)['content'].tolist()
    })

100%|██████████| 4121/4121 [00:02<00:00, 1505.92it/s]


In [20]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 3692.99it/s]


## 10

In [21]:
n_negative = 10
np.random.seed(seed)

dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': document_df[document_df['source'] != row['source']].sample(n=n_negative, random_state=seed)['content'].tolist()
    })

100%|██████████| 4121/4121 [00:02<00:00, 1511.21it/s]


In [22]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_Negative={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 2932.72it/s]


# เลือก n ตัวจาก similarity score

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
embeddings = HuggingFaceEmbeddings(model_name='/project/lt200301-edubot/Capstone-TamTanai/models/multilingual-e5-large',
                                   model_kwargs={"device": 'cuda'})

/tmp/ipykernel_1180/863869584.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name='/project/lt200301-edubot/Capstone-TamTanai/models/multilingual-e5-large',


In [24]:
document_df['embedding_vector'] = document_df['content'].progress_apply(embeddings.embed_query)
document_df

100%|██████████| 567/567 [00:21<00:00, 26.43it/s]


,source,content,embedding_vector
0,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...,"[0.006689948961138725, 0.01954878680408001, 0...."
1,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...,"[0.015548941679298878, 0.011010197922587395, 0..."
2,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...,"[0.009028371423482895, 0.016535772010684013, 0..."
3,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย/พระราชบัญ...,พระราชบัญญัติว่าด้วยการปรับเป็นพินัย พ.ศ. 2565...,"[0.01804278790950775, 0.01859115995466709, 0.0..."
4,พระราชบัญญัติจราจรทางบก/พระราชบัญญัติจราจรทางบ...,พระราชบัญญัติจราจรทางบก (ฉบับที่ 13) พ.ศ. 2565...,"[0.05025724321603775, -0.002424475969746709, 0..."
...,...,...,...
562,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 4 (วิธีก...,"[0.01567177101969719, 0.017037099227309227, 0...."
563,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...,"[0.0330001562833786, 0.012760871089994907, 0.0..."
564,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 2 (วิธีพ...,"[0.02422294206917286, -0.0014266790822148323, ..."
565,ประมวลกฎหมายวิธีพิจารณาความแพ่ง/ประมวลกฎหมายวิ...,ประมวลกฎหมายวิธีพิจารณาความแพ่ง - ภาค 1 (บททั่...,"[0.013988050632178783, 0.013145809061825275, 0..."


In [25]:
def query_top_n_similar_documents(question, n, document_df=document_df, source=None):
    if source:
        filtered_document_df = document_df.query('source != @source')
    else:
        filtered_document_df = document_df.copy()
    question_embedding_vector = embeddings.embed_query(question)
    similarity_scores = cosine_similarity([question_embedding_vector], filtered_document_df['embedding_vector'].tolist())[0]
    return filtered_document_df.iloc[similarity_scores.argsort()[-n:][::-1]]['content'].tolist()


def multithread_query_top_n_similar_documents(question, n, document_df=document_df, source=None):
    similar_documents = query_top_n_similar_documents(question=question, n=n, document_df=document_df, source=source)
    return {
        'query': question,
        'pos': document_df[document_df['source'] == source]['content'].tolist(),
        'neg': similar_documents
    }

In [26]:
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

## 5

In [73]:
n_negative = 5

dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': query_top_n_similar_documents(question=row['question'], n=n_negative, source=row['source'])
    })

100%|██████████| 4121/4121 [03:04<00:00, 22.32it/s]


In [74]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 4959.34it/s]


## 10

In [82]:
n_negative = 10

dataset = []
with ThreadPoolExecutor(max_workers=20) as executor:
    for data in tqdm(executor.map(multithread_query_top_n_similar_documents, df['question'], repeat(n_negative),
                                  repeat(document_df), df['source']), total=df.shape[0]):
        dataset.append(data)

100%|██████████| 4121/4121 [03:16<00:00, 21.01it/s]


In [83]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 2828.48it/s]


## 20

In [84]:
n_negative = 20

dataset = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for data in tqdm(executor.map(multithread_query_top_n_similar_documents, df['question'], repeat(n_negative),
                                  repeat(document_df), df['source']), total=df.shape[0]):
        dataset.append(data)

100%|██████████| 4121/4121 [03:08<00:00, 21.85it/s]


In [85]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:00<00:00, 1519.71it/s]


## 50

In [86]:
n_negative = 50

dataset = []
with ThreadPoolExecutor(max_workers=2) as executor:
    for data in tqdm(executor.map(multithread_query_top_n_similar_documents, df['question'], repeat(n_negative),
                                  repeat(document_df), df['source']), total=df.shape[0]):
        dataset.append(data)

100%|██████████| 4121/4121 [03:00<00:00, 22.79it/s]


In [87]:
train_dataset, validate_dataset = train_test_split(dataset, test_size=0.2, random_state=seed)

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 825/825 [00:01<00:00, 633.78it/s]


# เลือก n ตัวจาก similarity score และ keyword search

In [6]:
from pythainlp import word_tokenize, pos_tag
from pythainlp.corpus.common import thai_stopwords
import nltk
from nltk.corpus import stopwords
import re


if os.name != "nt":
    os.environ["TOKENIZERS_PARALLELISM"] = "false"


key_tags = ["NCMN", "NCNM", "NPRP", "NONM", "NLBL", "NTTL"]

thaistopwords = list(thai_stopwords())
nltk.download("stopwords")


def remove_stopwords(text):
    res = [
        word.lower()
        for word in text
        if (word not in thaistopwords and word not in stopwords.words())
    ]
    return res


def keyword_search(question, idf):
    tokens = word_tokenize(question, engine="newmm", keep_whitespace=False)
    pos_tags = pos_tag(tokens)
    noun_pos_tags = []
    for e in pos_tags:
        if e[1] in key_tags:
            noun_pos_tags.append(e[0])
    noun_pos_tags = remove_stopwords(noun_pos_tags)
    if idf:
        noun_pos_tags = [word for word in noun_pos_tags if word != 'มาตรา' or 'มาตรา' not in word]
    noun_pos_tags = list(set(noun_pos_tags))
    return noun_pos_tags


def find_case_number(text):
    pattern = re.compile(r"(?<!\d)(\d{1,5}/\d{4})(?!\d)")
    match = re.findall(pattern, text)
    if pattern.search(text) and all(e in mapper.values() for e in match):
        return [True, match]
    else:
        return [False, ""]


def find_law_number(text):
    pattern = re.compile(r'มาตรา ?(\d+(?:/\d+)?)\s*(ทวิ|ตรี|จัตวา|เบญจ|ฉ|สัตต|อัฏฐ|นว|ทศ|เอกาทศ|ทวาทศ|เตรส|จตุทศ|ปัณรส|โสฬส|สัตตรส|อัฏฐารส)?')
    match = re.findall(pattern, text)
    result = []
    for m in match:
        result.append(f'มาตรา {m[0]} {m[1]}'.strip())
    if result:
        return [True, result]
    else:
        return [False, ""]


def keyword_matcher(content, keywords):
    matched_keywords = []
    for keyword in keywords:
        pattern = re.compile(re.escape(keyword))
        if pattern.search(content):
            matched_keywords.append(keyword)
    return matched_keywords


def filter_docs_by_keywords(documents, keywords, question, law_number=True):
    if not keywords:
        return [], []
    filtered_docs = []
    matches = []
    found_law_number = False
    if law_number:
        found_law_number, law_numbers = find_law_number(question)
    for document in documents:
        if found_law_number:
            for law_number in law_numbers:
                pattern = re.compile(re.escape(law_number))
                if pattern.search(document):
                    matched_keywords = keyword_matcher(document, keywords)
                    if len(matched_keywords) >= min(2, len(keywords)):
                        matches.append(matched_keywords)
                        filtered_docs.append(document)
                        continue
        else:
            matched_keywords = keyword_matcher(document, keywords)
            if len(matched_keywords) >= min(2, len(keywords)):
                matches.append(matched_keywords)
                filtered_docs.append(document)
    return filtered_docs, matches


def random_n_keyword_detected_documents(question, n, document_df=document_df, source=None):
    if source:
        filtered_document_df = document_df.query('source != @source')
    else:
        filtered_document_df = document_df.copy()
    keywords = keyword_search(question, idf=True)
    filtered_docs, _ = filter_docs_by_keywords(filtered_document_df['content'].tolist(), keywords, question, law_number=True)
    np.random.seed(seed)
    if filtered_docs:
        return np.random.choice(filtered_docs, size=n, replace=n>len(filtered_docs)).tolist()
    else:
        return []


def multithread_random_n_keyword_detected_documents(question, n, document_df=document_df, source=None):
    keyword_detected_documents = random_n_keyword_detected_documents(question=question, n=n, document_df=document_df, source=source)
    return {
        'query': question,
        'pos': document_df[document_df['source'] == source]['content'].tolist(),
        'neg': keyword_detected_documents
    }

[nltk_data] Error loading stopwords: <urlopen error [Errno 101]
[nltk_data]     Network is unreachable>


## 5

In [64]:
n_negative = 5

dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': query_top_n_similar_documents(question=row['question'], n=n_negative, source=row['source'])
    })
    keyword_detected_documents = random_n_keyword_detected_documents(question=row['question'], n=n_negative, source=row['source'])
    if keyword_detected_documents:
        dataset.append({
            'query': row['question'],
            'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
            'neg': keyword_detected_documents
        })

100%|██████████| 4121/4121 [06:35<00:00, 10.42it/s]


In [75]:
queries = []
for d in dataset:
    if d['query'] not in queries:
        queries.append(d['query'])
train_queries, validate_queries = train_test_split(queries, test_size=0.2, random_state=seed)
train_dataset = [d for d in dataset if d['query'] in train_queries]
validate_dataset = [d for d in dataset if d['query'] in validate_queries]

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 1620/1620 [00:00<00:00, 4361.83it/s]


## เพิ่ม negative similar document จากผลของ reranker

In [15]:
from FlagEmbedding import FlagReranker

/home/shanwibo/.conda/envs/retrieval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
n_negative = 5

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path) as file:
    train_dataset = [json.loads(element) for element in list(file)]

In [16]:
model_path = f'/project/lt200301-edubot/Capstone-TamTanai/models/bge-reranker-v2-m3-finetune-with_similar={n_negative}_keyword={n_negative}'
reranker = FlagReranker(model_path, use_fp16=True)

----------using 4*GPUs----------


In [53]:
dataset = []
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    similar_documents = query_top_n_similar_documents(question=row['question'], n=40, source=row['source'])
    input_data = [[row['question'], doc] for doc in similar_documents]
    reranking_scores = reranker.compute_score(input_data, normalize=True)
    dataset.append({
        'query': row['question'],
        'pos': document_df[document_df['source'] == row['source']]['content'].tolist(),
        'neg': np.array(similar_documents)[np.array(reranking_scores).argsort()[-n_negative:][::-1]].tolist()
    })

100%|██████████| 4121/4121 [14:37<00:00,  4.70it/s]


In [76]:
dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_train.jsonl'
dataset_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(dataset_path) as file:
    train_dataset = [json.loads(element) for element in list(file)]

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_validate.jsonl'
dataset_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(dataset_path) as file:
    validate_dataset = [json.loads(element) for element in list(file)]

In [77]:
len(train_dataset), len(validate_dataset)

(6513, 1620)

In [78]:
train_queries = set([data['query'] for data in train_dataset])
validate_queries = set([data['query'] for data in validate_dataset])

In [79]:
for data in dataset:
    if data['query'] in train_queries:
        train_dataset.append(data)
    elif data['query'] in validate_queries:
        validate_dataset.append(data)

In [83]:
len(train_dataset), len(validate_dataset)

(9809, 2445)

In [84]:
dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_2nd_train.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(train_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

dataset_name = f'reranker_training_dataset_with_similar={n_negative}_keyword={n_negative}_2nd_validate.jsonl'
output_path = f'/project/lt200301-edubot/Capstone-TamTanai/reranker_training_dataset/{dataset_name}'
with open(output_path, 'w', encoding='utf-8') as file:
    for data in tqdm(validate_dataset):
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

100%|██████████| 2445/2445 [00:00<00:00, 4181.27it/s]
